In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Pytorch_Custom_Predict.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/custom_prediction_routines/SDK_Pytorch_Custom_Predict.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>

## 概述

本教程演示如何使用Vertex AI SDK构建一个自定义容器，该容器使用自定义预测例程模型服务器来提供在Vertex AI预测上服务的PyTorch模型。



### 数据集

本教程使用R.A. Fisher的鸢尾花数据集，这是一个小数据集，非常适合尝试机器学习技术。每个实例具有四个数值特征，这些特征是一朵花的不同测量值，以及一个目标标签，将其标记为三种类型之一的鸢尾花：山鸢尾，变色鸢尾或维吉尼亚鸢尾。

本教程使用[鸢尾花数据集](https://archive.ics.uci.edu/ml/datasets/iris)。

### 目标

目标是：
- 训练一个模型，使用一朵花的测量值作为输入来预测它是哪种类型的鸢尾花。
- 保存模型。
- 使用Vertex AI SDK中的自定义预测例程功能构建自定义PyTorch服务容器进行自定义预处理。
- 在本地测试构建的容器。
- 上传并部署自定义容器到Vertex Prediction。

本教程着重于使用Vertex AI部署此模型，而不是模型本身的设计。

### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI

了解更多关于[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing
Calculator ](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

设置本地开发环境

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

* Docker
* Git
* Google Cloud SDK（gcloud）
* Python 3
* virtualenv
* 在使用 Python 3 的虚拟环境中运行的 Jupyter 笔记本

Google Cloud 的 [设置 Python 开发环境](https://cloud.google.com/python/setup) 指南和 [Jupyter 安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了简化的说明：

1. [安装和初始化 Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

4. 要安装 Jupyter，在终端 shell 中的命令行中运行 `pip install jupyter`。

5. 要启动 Jupyter，在终端 shell 中的命令行中运行 `jupyter notebook`。

6. 在 Jupyter Notebook 仪表板中打开此笔记本。

安装额外的软件包

安装笔记本环境中尚未安装的额外软件包依赖项，例如NumPy、Scikit-learn、FastAPI、Uvicorn和joblib。

In [ ]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
pandas
torch==1.11.0
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

您部署的模型将预先安装了一个与您的笔记本环境不同的依赖项集合。您不应该假设因为在笔记本中工作正常，所以在模型中也会正常工作。相反，我们将通过在requirements.txt中列出它们并使用`pip install`来在笔记本中安装完全相同的依赖项来非常明确地说明模型的依赖关系。请注意，当然，有可能我们在requirements.txt中错过了笔记本中已经存在的依赖项。如果是这种情况，事情将在笔记本中运行，但不会在模型中运行。为了防范这种情况，我们将在部署到云端之前在本地测试模型。

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

### 重启内核

在安装了额外的软件包之后，您需要重新启动笔记本内核，这样它才能找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

在这个示例中设置日志记录

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得一个 $300 的免费信用额用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以 `!` 或 `%` 开头的行作为 shell 命令，并可以将 Python 变量插入这些命令中以 `$` 或 `{}`。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。下面是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在 Vertex AI 中使用多区域存储桶进行训练。并非所有区域都为所有 Vertex AI 服务提供支持。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### 配置项目和资源名称

如果你在一个直播教程会话中，你可能会使用一个共享的测试账号或项目。为了避免用户在创建的资源之间发生名称冲突，你可以为每个实例会话创建一个时间戳，并将其附加到你在此教程中创建的资源的名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

配置GCP资源名称。

In [ ]:
MODEL_ARTIFACT_DIR = "pytorch-cpr-model-sdk"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sdk"  # @param {type:"string"}
IMAGE = "pytorch-cpr-server-sdk"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "pytorch-cpr-model-sdk"  # @param {type:"string"}

`区域` - 在笔记本的其余部分操作中使用。请确保选择一个[Cloud Vertex AI 服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability)。您不能使用多区域存储存储桶进行 Vertex AI 的预测。

`MODEL_ARTIFACT_DIR` - Cloud Storage 存储桶中模型工件的文件夹路径，例如："my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的工件库的名称。

`IMAGE` - 将被推送的容器镜像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

### 创建一个云存储桶

**以下步骤都是必需的，无论您使用的是哪种笔记本环境。**

要在不重新构建容器的情况下更新您的模型工件，您必须将您的模型工件和任何自定义代码上传到云存储。

在下方设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

设置目录

决定将您所有自定义代码放在哪个目录。

In [ ]:
USER_SRC_DIR = "src_dir_pytorch"  # @param {type:"string"}

决定将你训练好的模型放在哪个目录下。

In [ ]:
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}

In [ ]:
%mkdir $USER_SRC_DIR
%mkdir $LOCAL_MODEL_ARTIFACTS_DIR

### 下载鸢尾花数据
在这个例子中，我们想要为简单的[鸢尾花数据集](https://archive.ics.uci.edu/ml/datasets/iris)构建一个分类器。所以首先，我们需要将数据 csv 文件下载到本地。

In [ ]:
DATA_DIR = "data_iris"

%mkdir $DATA_DIR

LOCAL_DATA_FILE = f"{DATA_DIR}/iris.csv"

In [ ]:
from urllib.request import urlretrieve

urlretrieve(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
    LOCAL_DATA_FILE,
)

### 构建一个PyTorch神经网络分类器

确保已[安装](https://pytorch.org/get-started/locally/)PyTorch软件包。

In [ ]:
import torch
from torch.autograd import Variable

print("PyTorch Version: {}".format(torch.__version__))

#### 第一步。加载数据

在这一步中，我们将:
1. 将数据加载到Pandas数据框中。
1. 将类特征（物种）从字符串转换为数值指示器。
1. 将数据框分割为输入特征（xtrain）和目标特征（ytrain）。

In [ ]:
import pandas as pd

CLASS_VOCAB = ["setosa", "versicolor", "virginica"]

datatrain = pd.read_csv(
    LOCAL_DATA_FILE,
    names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"],
)

# change string value to numeric
datatrain.loc[datatrain["species"] == "Iris-setosa", "species"] = 0
datatrain.loc[datatrain["species"] == "Iris-versicolor", "species"] = 1
datatrain.loc[datatrain["species"] == "Iris-virginica", "species"] = 2
datatrain = datatrain.apply(pd.to_numeric)

# change dataframe to array
datatrain_array = datatrain.values

# split x and y (feature and target)
xtrain = datatrain_array[:, :4]
ytrain = datatrain_array[:, 4]

input_features = xtrain.shape[1]
num_classes = len(CLASS_VOCAB)

print("Records loaded: {}".format(len(xtrain)))
print("Number of input features: {}".format(input_features))
print("Number of classes: {}".format(num_classes))

步骤2.设置模型参数
您可以尝试不同值的**隐藏单元**或**学习率**。

In [ ]:
HIDDEN_UNITS = 10
LEARNING_RATE = 0.1

#### 第三步。定义PyTorch NN模型
在这里，我们构建一个具有一个隐藏层的神经网络，并使用Softmax输出层进行分类。

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(input_features, HIDDEN_UNITS),
    torch.nn.Sigmoid(),
    torch.nn.Linear(HIDDEN_UNITS, num_classes),
    torch.nn.Softmax(),
)

loss_metric = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

步骤4：训练模型
我们将训练该模型进行**num_epoch**次。

In [ ]:
NUM_EPOCHS = 10000

for epoch in range(NUM_EPOCHS):

    x = Variable(torch.Tensor(xtrain).float())
    y = Variable(torch.Tensor(ytrain).long())
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_metric(y_pred, y)
    loss.backward()
    optimizer.step()
    if (epoch) % 1000 == 0:
        print(
            "Epoch [{}/{}] Loss: {}".format(
                epoch + 1, NUM_EPOCHS, round(loss.item(), 3)
            )
        )

print("Epoch [{}/{}] Loss: {}".format(epoch + 1, NUM_EPOCHS, round(loss.item(), 3)))

第五步。保存模型

In [ ]:
LOCAL_MODEL_FILE = f"{LOCAL_MODEL_ARTIFACTS_DIR}/model.pt"

torch.save(model, LOCAL_MODEL_FILE)

步骤6. 测试加载的模型进行预测

In [ ]:
iris_classifier = torch.load(LOCAL_MODEL_FILE)


def predict_class(instances):
    instances = torch.Tensor(instances)
    output = iris_classifier(instances)
    _, predicted = torch.max(output, 1)
    return predicted


predicted = predict_class(xtrain[0:5])
print([CLASS_VOCAB[class_index] for class_index in predicted])

### 上传模型工件和自定义代码到云存储

在部署模型进行服务之前，Vertex AI 需要访问以下在云存储中的文件：

* `model.pt`（模型工件）

运行以下命令以上传您的文件：

In [ ]:
!gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

使用CPR模型服务器构建定制的服务容器。

训练和保存了模型和处理器之后，现在是构建自定义服务容器的时候了。通常构建服务容器需要编写模型服务器代码。但是，通过自定义预测例程功能，Vertex AI Prediction提供了一个可以直接使用的模型服务器。

自定义服务容器包含以下3个代码片段：
1. [模型服务器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/model_server.py)
    * 托管模型的HTTP服务器。
    * 负责设置路由/端口等。
1. [请求处理程序](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)
    * 负责处理请求的Web服务器方面，例如反序列化请求主体，序列化响应，设置响应头等。
    * 在这个例子中，我们将使用SDK提供的默认Handler `google.cloud.aiplatform.prediction.handler.PredictionHandler`。
1. [预测器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)
    * 负责处理预测请求的ML逻辑。

这三个部分可以根据自定义容器的要求进行定制。在这个例子中，我们只会实现`预测器`。

[`预测器`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)必须实现以下接口：

```
from abc import ABC, abstractmethod
from typing import Any


class Predictor(ABC):
    """自定义预测例程的预测器类的接口。
    预测器负责处理预测请求的ML逻辑。
    具体而言，预测器必须定义：
    (1) 如何将用于预测的所有模型文件加载到内存中。
    (2) 预测时应执行的逻辑。
    当使用默认的PredictionHandler时，将如下所示调用预测器：
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """

    def __init__(self):
        return

    @abstractmethod
    def load(self, artifacts_uri: str) -> None:
        """加载模型文件。
        Args:
            artifacts_uri (str):
                必填。环境变量AIP_STORAGE_URI的值。
        """
        pass

    def preprocess(self, prediction_input: Any) -> Any:
        """在进行预测之前预处理预测输入。
        Args:
            prediction_input (Any):
                必填。需要预处理的预测输入。
        返回:
            预处理的预测输入。
        """
        return prediction_input

    @abstractmethod
    def predict(self, instances: Any) -> Any:
        """执行预测。
        Args:
            instances (Any):
                必填。用于执行预测的实例。
        返回:
            预测结果。
        """
        pass

    def postprocess(self, prediction_results: Any) -> Any:
        """后处理预测结果。
        Args:
            prediction_results (Any):
                必填。预测结果。
        返回:
            后处理的预测结果。
        """
        return prediction_results
```

首先，实现一个自定义的`Predictor`，它可以加载预处理器和模型。预处理器和模型将在`predict`时使用。

自定义预测程序支持一种在本地运行容器来测试图像的方法。在本地测试图像时，您可以传递一个GCS路径或一个本地路径。
- 如果传递一个GCS路径，则需要设置凭据。
- 如果想通过传递本地路径进行测试，您需要在`Predictor`中支持远程和本地加载您的模型。

Vertex SDK提供了一个[`download_model_artifacts`函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)，帮助您从GCS路径或本地路径下载模型工件。请参阅下面`load`函数中的示例。

In [ ]:
%%writefile $USER_SRC_DIR/predictor.py

import pandas as pd
import pickle
import torch
from typing import Dict

from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils


class CustomPyTorchPredictor(Predictor):
    
    def __init__(self):
        self._class_names = ["setosa", "versicolor", "virginica"]
    
    def load(self, artifacts_uri: str):
        """Loads the model artifacts."""
        prediction_utils.download_model_artifacts(artifacts_uri)

        self._model = torch.load("model.pt")

    def preprocess(self, prediction_input: Dict) -> torch.Tensor:
        instances = prediction_input["instances"]
        data = pd.DataFrame(instances).values
        return torch.Tensor(data)

    @torch.inference_mode()
    def predict(self, instances: torch.Tensor) -> torch.Tensor:
        """Performs prediction."""
        outputs = self._model(instances)
        _ , predicted = torch.max(outputs, 1)
        return predicted

    def postprocess(self, prediction_results: torch.Tensor) -> Dict:
        return {"predictions": [self._class_names[class_num] for class_num in prediction_results]}

构建自定义容器时，我们还需要编写一个启动模型服务器的图像入口点。但是，使用自定义预测例程功能，您不再需要编写入口点。Vertex SDK 将使用您提供的自定义预测程序填充入口点。

将要安装在镜像中的来源目录的依赖关系写入。

In [ ]:
!cp requirements.txt $USER_SRC_DIR/requirements.txt

构建并推送容器到Artifact Registry。

### 构建你的自定义容器

要构建一个自定义的镜像，需要一个 Dockerfile，在其中你需要实现镜像的外观。但是，利用自定义预测例程功能，Vertex SDK 也会为你生成 Dockerfile 并构建镜像。

默认情况下，使用 `python:3.7` 作为基础镜像。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src_dir_pytorch.predictor import \
    CustomPyTorchPredictor  # Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CustomPyTorchPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

您可以查看已构建图像的服务容器规格。

In [ ]:
local_model.get_serving_container_spec()

存储测试实例

要了解有关在 JSON 中格式化输入实例的更多信息，请阅读[文档。](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

### 在本地运行和测试容器

自定义预测例程提供两种下载模型工件以在本地测试图像的方式。
1. 本地路径。
2. GCS路径。

要使用本地路径，您的“预测器”需要支持加载模型的两种方式，以便在本地进行测试并部署到Vertex AI预测服务。SDK提供了一个方法`download_model_artifacts`来支持这两种方式，您可以在您的“预测器”的`load`函数中调用[prediction_utils.py](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)。

如果您想用GCS路径在本地测试图像，您需要按照指示设置凭据。

设置凭证

设置凭证只是为了在本地使用GCS路径运行自定义服务容器时才需要。设置凭证是为了执行`Predictor`的`load`函数，该函数会从Google Cloud Storage下载模型工件。

要在项目中访问Google Cloud Storage，您需要通过以下方式之一设置凭据：

1. 用户帐号

2. 服务帐号

您可以在[这里](https://cloud.google.com/docs/authentication#principals)了解更多关于上述每种帐号的信息。

选项1. 使用Google用户凭据

首先，授权Google认证库以使用Google用户凭据访问云平台。这一步涉及一个交互式提示，需要用户输入。如果您使用的是非交互式shell，您应该打开一个终端来运行这个命令。请参考[这里](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的机器上运行下面的命令，它会提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动Web浏览器的机器，例如笔记本电脑或工作站，才可以运行此命令。在使用浏览器完成认证步骤后，您需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在支持Web浏览器的机器上运行此命令，则无需使用单独的机器。

此命令将把凭据文件打印为`凭据保存到文件：[CREDENTIALS_FILE]`。

In [ ]:
!gcloud auth application-default login

指定凭证文件，该文件为json格式文件，请作为路径。

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

接下来，授权gcloud使用Google用户凭据访问云平台。用户凭据需要在项目中具有`setIamPolicy`权限。更多详细信息请参见[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

与上一步类似，这也需要用户输入。先前步骤的说明同样适用于这里。

In [ ]:
!gcloud auth login

给用户帐户授予角色。 使用您在gcloud auth步骤中使用的电子邮件地址，例如 `myemail@gmail.com`。 由于我们将使用此用户帐户从Google云存储下载模型工件，因此我们授予用户帐户**Storage Object Viewer**角色。 有关更多详细信息，请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

使用项目ID初始化Vertex AI SDK，此操作需要用户凭据。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

选项2。使用谷歌服务帐号凭证

如果您已经完成了上述选项1，请跳过此步骤。如果尚未启用，请先启用 IAM API。

In [ ]:
!gcloud services enable iam.googleapis.com

请指定在您的项目中应该是唯一的服务账户名称。

In [ ]:
SERVICE_ACCOUNT = "[SERVICE_ACCOUNT]"  # @param {type:"string"}

创建服务账户。

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

授权gcloud使用Google用户凭据访问Cloud平台。用户凭据需要在项目中具有`setIamPolicy`权限。更多详细信息请查看[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

这一步涉及一个交互式提示，需要用户输入。如果您正在使用非交互式shell，您应该打开一个终端来运行该命令。查看[这里](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的机器上运行下面的命令，它会提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动Web浏览器的机器，例如笔记本电脑或工作站，才允许运行此命令。完成使用浏览器的身份验证步骤后，您需要将验证代码复制粘贴回原始终端以继续登录流程。如果您在具有网络浏览器支持的机器上运行此命令，则无需使用单独的机器。

In [ ]:
!gcloud auth login

授予服务账号角色。由于我们将使用此服务账号从Google Cloud Storage下载模型文件，我们授予服务账号"Storage Object Viewer"角色。有关更多详细信息，请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

指定服务账号密钥文件名。

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

生成服务帐号密钥文件。这将在本地运行自定义服务容器时使用。

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

### 在本地运行并发送请求给容器

有了自定义预测例程功能，很容易在本地测试容器。

在这个例子中，容器执行一个预测请求和一个健康检查。

选项1. 传递本地路径

此选项要求“预测器”中的“加载”功能支持从GCS路径和本地路径加载模型工件。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应和内容。

In [ ]:
health_check_response, health_check_response.content

也打印出所有容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

选项2. 通过GCS路径

如果您想要通过GCS路径，您需要在上一步中设置好凭据，并在运行容器时传递凭据的路径。服务账号应具有 **Storage Object Viewer** 权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
predict_response, predict_response.content

打印出健康检查的响应及其内容。

In [ ]:
health_check_response, health_check_response.content

还打印出所有的容器日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

将容器推送到制件存储库

配置Docker以访问制件存储库。然后将您的容器映像推送到您的制件存储库存储库。

In [ ]:
!gcloud services list

如果 `artifactregistry.googleapis.com` 在您的项目中未启用，请在继续之前启用该API。

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

使用SDK推送图片。

In [ ]:
local_model.push_image()

部署到Vertex AI

### 上传自定义容器模型

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

使用LocalModel实例来上传模型。它会自动填充容器规范供您使用。

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

部署模型在Vertex AI上
完成这一步后，模型将部署并准备好进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送预测

使用Python SDK

In [ ]:
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

### 使用REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### 使用 gcloud 命令行界面

In [ ]:
!gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI